In [20]:
import random, os
from pwn import *
from sympy import symbols, Matrix
import numpy as np


# ma trận Vandermonde -  Alexandre-Théophile Vandermonde

p = 1009

def gen(degree, secret):
    poly = [random.randrange(0, p) for _ in range(degree + 1)]
    index = random.randint(0, degree)
    
    poly[index] = secret
    return poly

def poly_eval(f, x):
	return sum(c * pow(x, i, p) for i, c in enumerate(f)) % p

# t = 10 
# x = 3
# secret = 125

# s = gen(t, secret)

# for i in range(t):
#     x = x + 1
#     print(f"poly_eval({s}, {x}) = {poly_eval(s, x)}")

In [ ]:
import random, os
from pwn import *
from sympy import symbols, Matrix
import numpy as np

def vandermonde(xs):
    n = len(xs)
    return Matrix([[xs[i]**j for j in range(n)] for i in range(n)])


# =======================================================================
##### START PROGRAM
# =======================================================================
t = 20
p = 2 ** 256 - 189

V = vandermonde([i+1 for i in range(t)])
V_inv = V.inv()

port = 1337
host = 'ssss.chals.sekai.team'
r = remote(host, port, ssl=True, sni=host)
prev_coef = []
for _ in range(2):
    # Send t:
    r.sendline(str(t).encode())

    # Start loop x:
    matrix_mod = []
    for i in range(t):
        # Send x:
        r.sendline(str(i+1).encode())     # 0<i<p
        fi = r.recvline(keepends=False, timeout=10).strip().decode()
        matrix_mod.append([int(fi)])

    matrix_mod = Matrix(matrix_mod)
    result = V_inv * matrix_mod

    coef_arr = result.applyfunc(lambda x: x % p)
    this_coef = [int(v) % p for v in coef_arr]
    print(this_coef)
    
    if len(prev_coef) == 0:
        prev_coef = this_coef.copy()
        r.sendline(str(prev_coef[5]).encode())
        temp = r.recvline(keepends=False, timeout=10).strip().decode()
        print(temp)
        
    else: 
        for val in this_coef:
            for res in prev_coef:
                if val == res:
                    r.sendline(str(val).encode())
                    flag = r.recvline(keepends=False, timeout=10).strip().decode()
                    print(flag)
                    break
        r.sendline(str(this_coef[5]).encode())
        temp = r.recvline(keepends=False, timeout=10).strip().decode()
        print(temp)   
            
# r.interactive()

[x] Opening connection to ssss.chals.sekai.team on port 1337
[x] Opening connection to ssss.chals.sekai.team on port 1337: Trying 34.133.17.18
[+] Opening connection to ssss.chals.sekai.team on port 1337: Done
[73743267020621864920610443901894181007418848617594108510084033123827657724273, 60449974506215250377684400327992345737010928242601194749621702907259329286526, 17557828074303810415984153209684464036750949276324001054302452235243123742972, 69405401056444897134123741397251182465674018284989196718782371456939469866209, 42311396232558482374238869388030484241061292156924903610084782964707066515501, 45676105533097335655351956806125181406223072425470719595854335071738925270089, 72721991652835590482359560703848291466896676603836093604494844241595741846605, 18962991976863520270324349378840169817461498554278780524807701930297614206356, 54721583646975173565434241945506772441240077751725785481523191065530829977996, 114000150945265973783744217270391184542816667818262209387974048502731479044855

In [37]:
p = 1009
FLAG = "HEHE"

def gen(degree, secret):
    poly = [random.randrange(0, p) for _ in range(degree + 1)]
    index = random.randint(0, degree)
    
    poly[index] = secret
    return poly

def poly_eval(f, x):
	return sum(c * pow(x, i, p) for i, c in enumerate(f)) % p

def challenge(secret):
	t = int(input())
	assert 20 <= t <= 50, "Number of parties not in range"

	f = gen(t, secret)

	for i in range(t):
		x = int(input())
		assert 0 < x < p, "Bad input"
		print(poly_eval(f, x))

	if int(input()) == secret:
		print(FLAG)
		exit(0)
	else:
		print(":<")

def gen(degree, secret):
	poly = [random.randrange(0, p) for _ in range(degree + 1)]
	index = random.randint(0, degree)

	poly[index] = secret
	return poly

def poly_eval(f, x):
	return sum(c * pow(x, i, p) for i, c in enumerate(f)) % p

secret = 102
choose = 0

noo_coef = []

for _ in range(1):
    t = 20
    x_lst = [(i+1) for i in range(t)]
    f = gen(t, secret)
    print(f"Poly: {f}")
    
    for i in range(t):
        x = x_lst[i]
        
        temp = poly_eval(f, x)
        noo_coef.append(temp)
        
    if choose == secret:
        print(FLAG)
        exit(0)
    else:
        print(":<")

V = Matrix([[pow(x, i, p) for i in range(t)] for x in range(1, t+1)])
V_inv = V.inv_mod(p)

noo_coef = Matrix(noo_coef)

res = V_inv * noo_coef

this_coef = [int(v) % p for v in res]
print(this_coef)

Poly: [743, 975, 398, 480, 910, 264, 26, 454, 474, 362, 460, 125, 743, 510, 105, 105, 273, 102, 821, 23, 1003]
:<
[601, 482, 241, 533, 850, 591, 172, 24, 764, 785, 943, 385, 9, 165, 558, 877, 932, 268, 404, 781]


#### **Bài toán Shamir’s Secret Sharing (SSS)**